In [1]:
import pandas as pd

data_path = "../data/clean_data.csv"
df = pd.read_csv(data_path)

df.head()

,business_name,author_name,text,rating,rating_category
0,Haci'nin Yeri - Yigit Lokantasi,Gulsum Akar,We went to Marmaris with my wife for a holiday...,5,taste
1,Haci'nin Yeri - Yigit Lokantasi,Oguzhan Cetin,During my holiday in Marmaris we ate here to f...,4,menu
2,Haci'nin Yeri - Yigit Lokantasi,Yasin Kuyu,Prices are very affordable. The menu in the ph...,3,outdoor_atmosphere
3,Haci'nin Yeri - Yigit Lokantasi,Orhan Kapu,Turkey's cheapest artisan restaurant and its f...,5,indoor_atmosphere
4,Haci'nin Yeri - Yigit Lokantasi,Ozgur Sati,I don't know what you will look for in terms o...,3,menu


In [2]:
# extract lengths of each review
df["review_length"] = df["text"].apply(len)
df.head()

,business_name,author_name,text,rating,rating_category,review_length
0,Haci'nin Yeri - Yigit Lokantasi,Gulsum Akar,We went to Marmaris with my wife for a holiday...,5,taste,680
1,Haci'nin Yeri - Yigit Lokantasi,Oguzhan Cetin,During my holiday in Marmaris we ate here to f...,4,menu,914
2,Haci'nin Yeri - Yigit Lokantasi,Yasin Kuyu,Prices are very affordable. The menu in the ph...,3,outdoor_atmosphere,173
3,Haci'nin Yeri - Yigit Lokantasi,Orhan Kapu,Turkey's cheapest artisan restaurant and its f...,5,indoor_atmosphere,63
4,Haci'nin Yeri - Yigit Lokantasi,Ozgur Sati,I don't know what you will look for in terms o...,3,menu,161


### using a smaller subset of the data, df2 to try feature engineering first

In [3]:
import spacy

df2 = df.head(30).copy()  # Always copy when modifying
nlp = spacy.load("en_core_web_sm")

tokens_list = []
for text in df2["text"]:
    doc = nlp(text)
    tokens_list.append([token.text for token in doc])

df2["tokens"] = tokens_list
df2

,business_name,author_name,text,rating,rating_category,review_length,tokens
0,Haci'nin Yeri - Yigit Lokantasi,Gulsum Akar,We went to Marmaris with my wife for a holiday...,5,taste,680,"[We, went, to, Marmaris, with, my, wife, for, ..."
1,Haci'nin Yeri - Yigit Lokantasi,Oguzhan Cetin,During my holiday in Marmaris we ate here to f...,4,menu,914,"[During, my, holiday, in, Marmaris, we, ate, h..."
2,Haci'nin Yeri - Yigit Lokantasi,Yasin Kuyu,Prices are very affordable. The menu in the ph...,3,outdoor_atmosphere,173,"[Prices, are, very, affordable, ., The, menu, ..."
3,Haci'nin Yeri - Yigit Lokantasi,Orhan Kapu,Turkey's cheapest artisan restaurant and its f...,5,indoor_atmosphere,63,"[Turkey, 's, cheapest, artisan, restaurant, an..."
4,Haci'nin Yeri - Yigit Lokantasi,Ozgur Sati,I don't know what you will look for in terms o...,3,menu,161,"[I, do, n't, know, what, you, will, look, for,..."
5,Haci'nin Yeri - Yigit Lokantasi,Arda Karaca,Generally good.,4,indoor_atmosphere,15,"[Generally, good, .]"
6,Haci'nin Yeri - Yigit Lokantasi,İrem Eren,What you see is 125 TL in total. It's a pretty...,5,taste,245,"[What, you, see, is, 125, TL, in, total, ., It..."
7,Haci'nin Yeri - Yigit Lokantasi,Nadia Salim,Delicious food at rock bottom prices. Friendly...,5,taste,70,"[Delicious, food, at, rock, bottom, prices, .,..."
8,Haci'nin Yeri - Yigit Lokantasi,Mehmet Eser,"Every time I go, I still experience the amazem...",5,outdoor_atmosphere,157,"[Every, time, I, go, ,, I, still, experience, ..."
9,Haci'nin Yeri - Yigit Lokantasi,Celal Ozer,The most f/p of all businesses I've seen.,5,indoor_atmosphere,41,"[The, most, f, /, p, of, all, businesses, I, '..."


### we can also train this model for our own needs but idk if it will be easy

In [4]:
from sentence_transformers import SentenceTransformer, util

# 1. Load model
model = SentenceTransformer("all-MiniLM-L6-v2")

# 2. Encode all texts and reference words
text_embeddings = model.encode(df2["text"].tolist(), convert_to_tensor=True)
reference_embeddings = model.encode(["restaurant", "food"], convert_to_tensor=True)

# 3. Calculate pairwise cosine similarities
similarity_matrix = util.cos_sim(text_embeddings, reference_embeddings)

print(similarity_matrix.shape)

# 4. For each text, get similarity scores to both references
similarity_scores = []
for i in range(len(df2)):
    # Get similarity scores for this text to both reference words
    scores = similarity_matrix[i].tolist()
    similarity_scores.append(scores)

df2["similarity_scores"] = similarity_scores
df2
# This will give you a list for each row: [similarity_to_restaurant, similarity_to_food]

/Users/lakshmi/Downloads/techjam/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


torch.Size([30, 2])


,business_name,author_name,text,rating,rating_category,review_length,tokens,similarity_scores
0,Haci'nin Yeri - Yigit Lokantasi,Gulsum Akar,We went to Marmaris with my wife for a holiday...,5,taste,680,"[We, went, to, Marmaris, with, my, wife, for, ...","[0.5075353980064392, 0.37266629934310913]"
1,Haci'nin Yeri - Yigit Lokantasi,Oguzhan Cetin,During my holiday in Marmaris we ate here to f...,4,menu,914,"[During, my, holiday, in, Marmaris, we, ate, h...","[0.4083253741264343, 0.30511540174484253]"
2,Haci'nin Yeri - Yigit Lokantasi,Yasin Kuyu,Prices are very affordable. The menu in the ph...,3,outdoor_atmosphere,173,"[Prices, are, very, affordable, ., The, menu, ...","[0.33979272842407227, 0.28572386503219604]"
3,Haci'nin Yeri - Yigit Lokantasi,Orhan Kapu,Turkey's cheapest artisan restaurant and its f...,5,indoor_atmosphere,63,"[Turkey, 's, cheapest, artisan, restaurant, an...","[0.477489173412323, 0.2990363538265228]"
4,Haci'nin Yeri - Yigit Lokantasi,Ozgur Sati,I don't know what you will look for in terms o...,3,menu,161,"[I, do, n't, know, what, you, will, look, for,...","[0.4761326313018799, 0.22589965164661407]"
5,Haci'nin Yeri - Yigit Lokantasi,Arda Karaca,Generally good.,4,indoor_atmosphere,15,"[Generally, good, .]","[0.1795356571674347, 0.23901411890983582]"
6,Haci'nin Yeri - Yigit Lokantasi,İrem Eren,What you see is 125 TL in total. It's a pretty...,5,taste,245,"[What, you, see, is, 125, TL, in, total, ., It...","[0.41842225193977356, 0.15947109460830688]"
7,Haci'nin Yeri - Yigit Lokantasi,Nadia Salim,Delicious food at rock bottom prices. Friendly...,5,taste,70,"[Delicious, food, at, rock, bottom, prices, .,...","[0.43825483322143555, 0.36904144287109375]"
8,Haci'nin Yeri - Yigit Lokantasi,Mehmet Eser,"Every time I go, I still experience the amazem...",5,outdoor_atmosphere,157,"[Every, time, I, go, ,, I, still, experience, ...","[0.083377406001091, 0.07767815887928009]"
9,Haci'nin Yeri - Yigit Lokantasi,Celal Ozer,The most f/p of all businesses I've seen.,5,indoor_atmosphere,41,"[The, most, f, /, p, of, all, businesses, I, '...","[0.29386070370674133, 0.13103997707366943]"


In [5]:
from transformers import pipeline


# Step 2: Load the pre-trained sentiment analysis pipeline
sentiment_analyzer = pipeline("sentiment-analysis")
labels = []
scores = []

for text in df2["text"]:
    result = sentiment_analyzer(text)[0]  # returns list of dicts
    label = result['label']
    score = result['score']
    labels.append(label)
    scores.append(score)

    print(f"Text: {text}")
    print(f" → Sentiment: {label} (confidence: {score:.4f})\n")

df2["sentiment"] = labels
df2["confidence score"] = scores
df2

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use mps:0


Text: We went to Marmaris with my wife for a holiday. We chose this restaurant as a place for dinner based on the reviews and because we wanted juicy food. When we first went there was a serious queue. You proceed by taking the food you want in the form of an open buffet. Both vegetable dishes and meat dishes were plentiful. There was also dessert for those who wanted it. After you get what you want you pay at the cashier. They don't go through cards they work in cash. There was a lot of food variety. And the food prices were unbelievably cheap. We paid only 84 TL for all the meals here. It included buttermilk and bread. But unfortunately I can't say it's too clean as a place..
 → Sentiment: NEGATIVE (confidence: 0.9952)

Text: During my holiday in Marmaris we ate here to fit the food. It's really good that the food is cheap and nice. Eating as much bread as you want is a big plus for those who are not satisfied without bread. It is a place that I can recommend to those who will go to 

,business_name,author_name,text,rating,rating_category,review_length,tokens,similarity_scores,sentiment,confidence score
0,Haci'nin Yeri - Yigit Lokantasi,Gulsum Akar,We went to Marmaris with my wife for a holiday...,5,taste,680,"[We, went, to, Marmaris, with, my, wife, for, ...","[0.5075353980064392, 0.37266629934310913]",NEGATIVE,0.995151
1,Haci'nin Yeri - Yigit Lokantasi,Oguzhan Cetin,During my holiday in Marmaris we ate here to f...,4,menu,914,"[During, my, holiday, in, Marmaris, we, ate, h...","[0.4083253741264343, 0.30511540174484253]",NEGATIVE,0.861798
2,Haci'nin Yeri - Yigit Lokantasi,Yasin Kuyu,Prices are very affordable. The menu in the ph...,3,outdoor_atmosphere,173,"[Prices, are, very, affordable, ., The, menu, ...","[0.33979272842407227, 0.28572386503219604]",POSITIVE,0.994443
3,Haci'nin Yeri - Yigit Lokantasi,Orhan Kapu,Turkey's cheapest artisan restaurant and its f...,5,indoor_atmosphere,63,"[Turkey, 's, cheapest, artisan, restaurant, an...","[0.477489173412323, 0.2990363538265228]",POSITIVE,0.999729
4,Haci'nin Yeri - Yigit Lokantasi,Ozgur Sati,I don't know what you will look for in terms o...,3,menu,161,"[I, do, n't, know, what, you, will, look, for,...","[0.4761326313018799, 0.22589965164661407]",POSITIVE,0.998186
5,Haci'nin Yeri - Yigit Lokantasi,Arda Karaca,Generally good.,4,indoor_atmosphere,15,"[Generally, good, .]","[0.1795356571674347, 0.23901411890983582]",POSITIVE,0.999745
6,Haci'nin Yeri - Yigit Lokantasi,İrem Eren,What you see is 125 TL in total. It's a pretty...,5,taste,245,"[What, you, see, is, 125, TL, in, total, ., It...","[0.41842225193977356, 0.15947109460830688]",POSITIVE,0.999794
7,Haci'nin Yeri - Yigit Lokantasi,Nadia Salim,Delicious food at rock bottom prices. Friendly...,5,taste,70,"[Delicious, food, at, rock, bottom, prices, .,...","[0.43825483322143555, 0.36904144287109375]",POSITIVE,0.999809
8,Haci'nin Yeri - Yigit Lokantasi,Mehmet Eser,"Every time I go, I still experience the amazem...",5,outdoor_atmosphere,157,"[Every, time, I, go, ,, I, still, experience, ...","[0.083377406001091, 0.07767815887928009]",POSITIVE,0.861084
9,Haci'nin Yeri - Yigit Lokantasi,Celal Ozer,The most f/p of all businesses I've seen.,5,indoor_atmosphere,41,"[The, most, f, /, p, of, all, businesses, I, '...","[0.29386070370674133, 0.13103997707366943]",POSITIVE,0.998013
